### 1. Add Huggingface Transformers and LLaMA Weight

#### 1.1 install hf transformers
    git clone https://github.com/huggingface/transformers.git
    cd transformers
    pip install ./

#### 1.2 Transcript LLaMA Weight to HF format
    # 7B
    python /home/linjw/codebase/preaction/3rdparty/transformers/src/transformers/models/llama/convert_llama_weights_to_hf.py --input_dir /home/linjw/codebase/preaction/data/gpt/llama_weights/LLaMA_ori/ --model_size 7B --output_dir /home/shizhenkun/codebase/preaction/data/gpt/llama_weights/llama-7b-hf

    # 13B
    python /home/linjw/codebase/preaction/3rdparty/transformers/src/transformers/models/llama/convert_llama_weights_to_hf.py --input_dir /home/linjw/codebase/preaction/data/gpt/llama_weights/LLaMA_ori/ --model_size 13B --output_dir /home/shizhenkun/codebase/preaction/data/gpt/llama_weights/llama-13b-hf

    # 30B
    python /home/linjw/codebase/preaction/3rdparty/transformers/src/transformers/models/llama/convert_llama_weights_to_hf.py --input_dir /home/linjw/codebase/preaction/data/gpt/llama_weights/LLaMA_ori/ --model_size 30B --output_dir /home/shizhenkun/codebase/preaction/data/gpt/llama_weights/llama-30b-hf

    #65B
    python /home/linjw/codebase/preaction/3rdparty/transformers/src/transformers/models/llama/convert_llama_weights_to_hf.py --input_dir /home/linjw/codebase/preaction/data/gpt/llama_weights/LLaMA_ori/ --model_size 65B --output_dir /home/shizhenkun/codebase/preaction/data/gpt/llama_weights/llama-65b-hf


### 2. Add Vicuna Lora Weight to LLaMA

    # 13B
    python -m fastchat.model.apply_delta \
        --base /home/linjw/codebase/preaction/data/gpt/llama_weights/llama-13b-hf \
        --target /home/linjw/codebase/preaction/data/gpt/fastchat_weight/vicuna/vicuna-13b \
        --delta /home/linjw/codebase/preaction/data/gpt/vicuna/vicuna-13b-delta-v1.1

### 3. Start Vicuna with Web Sever

#### 3.1 Single Model

    conda activate preaction
    cd /home/linjw/codebase/preaction/3rdparty/FastChat

    #start controller
    python -m fastchat.serve.controller --host 0.0.0.0 --port 10002

    #load model
    # python -m fastchat.serve.model_worker --model-path /home/linjw/codebase/preaction/data/gpt/fastchat_weight/vicuna/vicuna-13b  --controller http://127.0.0.1:10002 --num-gpus 1
    python -m fastchat.serve.model_worker --model-path /home/linjw/codebase/preaction/data/gpt/fastchat_weight/vicuna/vicuna-7b  --controller http://127.0.0.1:10002 --num-gpus 1
    #start web-ui
    python -m fastchat.serve.gradio_web_server --controller http://127.0.0.1:10002 --host banana.tibd.net --model-list-mode reload

### 4. Train Your Own Model

    torchrun --nproc_per_node=8 --master_port=20001 fastchat/train/train_mem.py \
        --model_name_or_path /home/linjw/codebase/preaction/data/gpt/fastchat_weight/vicuna/vicuna-7b  \
        --data_path /home/linjw/codebase/preaction/3rdparty/FastChat/playground/data/dummy.json \
        --bf16 True \
        --output_dir /home/linjw/codebase/preaction/data/gpt/finetuned/vicuna_dummy \
        --num_train_epochs 3 \
        --per_device_train_batch_size 2 \
        --per_device_eval_batch_size 2 \
        --gradient_accumulation_steps 16 \
        --evaluation_strategy "no" \
        --save_strategy "steps" \
        --save_steps 1200 \
        --save_total_limit 10 \
        --learning_rate 2e-5 \
        --weight_decay 0. \
        --warmup_ratio 0.03 \
        --lr_scheduler_type "cosine" \
        --logging_steps 1 \
        --fsdp "full_shard auto_wrap" \
        --fsdp_transformer_layer_cls_to_wrap 'LlamaDecoderLayer' \
        --tf32 True \
        --model_max_length 2048 \
        --gradient_checkpointing True \
        --lazy_preprocess True

### 5. Using Alpaca Lora to Train and Finetune

    CUDA_VISIBLE_DEVICES=1 python finetune.py \
    --base_model '/home/linjw/codebase/FastChat/data/fastchat_weight/vicuna' \
    --data_path '/home/linjw/codebase/FastChat/data/corpus/alpaca_data_cleaned_archive.json' \
    --output_dir '/home/linjw/codebase/FastChat/data/finetuned/bio' \
    --batch_size 128 \
    --micro_batch_size 4 \
    --num_epochs 3 \
    --learning_rate 1e-4 \
    --cutoff_len 512 \
    --val_set_size 2000 \
    --lora_r 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --lora_target_modules '[q_proj,k_proj,v_proj,o_proj]' \
    --train_on_inputs \
    --group_by_length